In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.metrics import confusion_matrix
import mysql.connector
from mysql.connector.cursor import MySQLCursor
from mysql.connector import errorcode

In [2]:
# TODO: better handle case when the table exists
def create_table(name, create, table, cursor):
    """Create a table in an SQL database.
    
    Warning: this function overrides any exisiting data!
    
    Attributes:
        name (str): Name of the table
        create (str): SQL statement for creating the table
        table (pd.DataFrame): The table to be added
        cursor (MySQLCursor): Cursor for MySQL database
    """
    try:
        cursor.execute(create)
    except mysql.connector.Error as err:
        print(err)
        cursor.execute("DROP TABLE %s;" % name)
        cursor.execute(create)
        
    cols = ', '.join(list(table.columns))
    vals = ', '.join(map((lambda x: '%%(%s)s' % x), list(table.columns)))
    insert = """INSERT INTO %s 
    (%s)
    VALUES (%s)
    """ % (name, cols, vals)

    for row in table.reset_index().iterrows():
        data = row[1].to_dict()
        try:
            cursor.execute(insert, data)
        except mysql.connector.Error as err:
            print(err)

In [3]:
# connect to database
cnx = mysql.connector.connect(user='root',
                              password='HWROBB01',
                              host='localhost',
                              database='umphbase')
cursor = cnx.cursor()

In [4]:
shows = pd.read_pickle('atu_database/shows.pickle')
shows['show_date'] = shows['show_date'].astype(str)

create_shows = """
CREATE TABLE shows (
    show_id BIGINT PRIMARY KEY,
    show_date TEXT NOT NULL,
    show_order TEXT NOT NULL,
    artist TEXT NOT NULL,
    venue_id TEXT NOT NULL,
    tour_name TEXT NOT NULL,
    show_notes LONGTEXT,
    opener TEXT,
    sound_check LONGTEXT
);
"""

# TODO: decide how to deal with dates
# TODO: Why are some nans not getting replaced?
create_table('shows', create_shows, shows, cursor)

1050 (42S01): Table 'shows' already exists
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'
1054 (42S22): Unknown column 'nan' in 'field list'


In [5]:
songs = pd.read_pickle('atu_database/songs.pickle')

create_songs = """
CREATE TABLE songs (
    song_id INT PRIMARY KEY,
    name TEXT NOT NULL,
    slug TEXT NOT NULL,
    original_artist TEXT NOT NULL,
    original BOOLEAN NOT NULL
);
"""

create_table('songs', create_songs, songs, cursor)

1050 (42S01): Table 'songs' already exists


In [6]:
venues = pd.read_pickle('atu_database/venues.pickle')

create_venues = """
CREATE TABLE venues (
    venue_id TEXT NOT NULL,
    venue_name TEXT NOT NULL,
    city TEXT NOT NULL,
    state TEXT,
    country TEXT NOT NULL
);
"""

create_table('venues', create_venues, venues, cursor)

1050 (42S01): Table 'venues' already exists


In [7]:
live_songs = pd.read_pickle('atu_database/live_songs.pickle')

create_live_songs = """
CREATE TABLE live_songs (
    live_song_id BIGINT PRIMARY KEY,
    show_id BIGINT NOT NULL,
    song_id INT NOT NULL,
    set_number TEXT NOT NULL,
    position INT NOT NULL,
    transition TEXT,
    footnote TEXT,
    jamchart BOOLEAN NOT NULL,
    jamchart_notes TEXT
);
"""

create_table('live_songs', create_live_songs, live_songs, cursor)

1050 (42S01): Table 'live_songs' already exists


In [8]:
# commit changes and close connection
cursor.close()
cnx.commit()
cnx.close()